# Assignment 1 - Calibrazione

## Emanuele Muzio - 0766230

Per questo primo assignment, viene richiesto di stimare:

1. I parametri intrinseci della camera (matrice K);
2. Posa della camera (parametri estrinseci R e t);
3. Pixel nelle immagini acquisite corrispondenti al pavimento.

Per i primi due punti è stato scelto l'algoritmo di calibrazione DLT, che consiste di 4 step e ci consente di:

1.1: Trovare le corrispondenze tra i punti 2D e 3D dell'immagine;

1.2: Derivare la matrice di camera P;

1.3: Usare la fattorizzazione QR per derivare K e R;

2.1: Trovare il vettore di traslazione _t_.

Viene richiesto inoltre di fare analizzare i risultati ottenuti confrontandoli con le implementazioni già esistenti nella libreria Python per la CV _OpenCV_, usando per il DLT un set di 6, 12 e 24 punti per le stime.

Dovremo infine proiettare i punti 3D sul pavimento, e testarne la bontà tramite MSE. In particolare, per testare la posa della camera, dovremo usare almeno 12 punti e usare una tecnica di **4-fold cross-validation** per misurare l'MSE medio nelle 4 pieghe/fold.  

### 1.1 Corrispondenze dei punti

![CUBE](cube.png)

Prepariamo per prima cosa due set di punti:
- Punti dell'oggetto, ovvero false coordinate 3D che hanno come punto di riferimento un angolo del cubo;
- Punti dell'immagine, ovvero coordinate nell'immagine dei relativi punti 3D.

Decidiamo inoltre quanti punti vogliamo usare per la calibrazione (più punti, calibrazione migliore?)

In [1]:
import dlt

points = 12 # 6, 12 o 24
objp, imgp = dlt.getPoints(points)

Inizializziamo i valori ottenibili da OpenCv

In [2]:
OPENCV_ret, OPENCV_K, OPENCV_dist, OPENCV_Rs, OPENCV_ts = dlt.OPENCVCalibrateCamera(points)

### 1.2 Matrice di camera P

Fatto questo procediamo a stimare la matrice di camera P di dimensioni _3x4_ usando le corrispondenze dei punti.
Il principio alla base di questo passaggio è quello per cui, sapendo che un punto dell'immagine è ottenuto dal prodotto tra un punto dell'oggetto e la matrice di camera, possiamo riscrivere il seguente sistema:

![Screen1](./screen/1.png)

In questo modo: 

![Screen2](./screen/2.png)

Quello che abbiamo adesso è un problema nella forma $Ap = 0$. Quello che possiamo fare per la matrice di proiezione, inoltre, è un'approssimazione quanto più corretta fino a un certo grado, per cui abbiamo due opzioni:
- Dividere per $p_3,_4$ tutti i parametri
- Aggiungere un vincolo tale per cui $||p||^2 = 1$

Nel secondo caso, dovremo trovare una p tale da avvicinarci quanto più possibile allo 0 e risolvere il problema di minimizzazione.

La soluzione è quindi applicare SVD alla matrice A, per cui $A = USV^T$.
Fatto questo, la matrice P risiederà nell'ultimo vettore di $V$ (andiamo a modificarne la forma per avere la nostra matrice normalizzata P 3x4).

In [3]:
P = dlt.estimateCameraMatrix(objp, imgp, points)
P = P/dlt.np.linalg.norm(P)
P = P.reshape(3,4)

### 1.3 Fattorizzazione RQ

Una volta trovata la matrice di proiezione, dobbiamo ricordarci che per le prime tre colonne di P, abbiamo:

![Screen3](./screen/3.png)

Ovvero cioè una matrice triangolare superiore per una matrice ortogonale, quindi una fattorizzazione RQ.
Una volta trovata quindi la matrice K (tramite fattorizzazione RQ troviamo R (=K) e Q (=R)).

In [4]:
K, R = dlt.getKR(P)

### 2.1 Vettore di traslazione

Possiamo procedere a calcolare $t = K^{-1}P_4$. 

In [5]:
t = dlt.getT(K, P)

## 3.1 Punti del pavimento

Procediamo adesso a trovare i punti del pavimento della nostra immagine.

Normalizziamo in primo luogo K affinchè in posizione K$_3_3$ ci sia 1.

Analizziamo la differenza tra la matrice K calcolata da noi e quella calcolata da OpenCV:

In [6]:
K = K/K[2,2]

print(
    'K', '\n\n', K, '\n\n', # Matrice calcolata ,
    'K OpenCV', '\n\n', OPENCV_K, '\n\n', # Matrice calcolata da OpenCV
    'Errore assoluto', '\n\n', abs(K - OPENCV_K), 
)

K 

 [[-1.1291652e+03  3.9733681e+01  1.4317920e+02]
 [-0.0000000e+00  1.0422750e+03 -3.3560303e+02]
 [-0.0000000e+00 -0.0000000e+00  1.0000000e+00]] 

 K OpenCV 

 [[ 1.07050242e+03  0.00000000e+00  6.62627768e+01]
 [ 0.00000000e+00  9.97867856e+02 -3.44741388e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]] 

 Errore assoluto 

 [[2199.66757814   39.73368073   76.91642244]
 [   0.           44.40716877    9.13836028]
 [   0.            0.            0.        ]]


All'aumentare dei punti utilizzati per i calcoli abbiamo una diminuzione visibile dell'errore.

Procediamo infine a proiettare i punti al suolo e all'analisi delle differenze, trasformando per prima cosa la matrice di rotazione R in un vettore di rotazione grazie alla funzione presente nella libreria di OpenCV _Rodrigues(R)_

In [7]:
rodr = dlt.cv.Rodrigues(R)

OPENCV_ground = dlt.getGround(OPENCV_Rs[0], OPENCV_ts[0], OPENCV_K, OPENCV_dist)
ground = dlt.getGround(rodr[0], t, K, None)

dlt.drawGround(OPENCV_ground, 'OpenCV Ground')
dlt.drawGround(ground, 'My Ground')

averageMse = dlt.KFoldCrossVal(ground, OPENCV_ground, 4)

print('Average MSE: \n\n', averageMse)

Average MSE: 

 218.85333333333335


Sperimentalmente, notiamo una variazione notevole dell'MSE medio al variare dei punti N presi in considerazione:

| N  | Average MSE |
|----|-------------|
| 6  | 165274.68   |
| 12 | 218.85      |
| 24 | 29.15       |

## Riferimenti

- W. Burger, 2016, Zhang’s Camera Calibration Algorithm: In-Depth Tutorial and Implementation. _Technical Report HGB16-05_.
- Slide Visione Artificiale